In [1]:
%load_ext autoreload
%autoreload 2

from kloppy import sportec
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from mplsoccer import Pitch
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm
pd.set_option('display.max_columns', None)  # 모든 열 출력
from unravel.soccer import KloppyPolarsDataset

import os 
import pickle
import sys

sys.path.append(os.path.abspath(".."))
from pressing_intensity import CustomPressingIntensity
from visualization import plot_single_frame_positions, plot_window_frame_positions
from config import TEAMNAME2ID, TEAMID2NAME

coordinates = "secondspectrum"

2025-06-14 01:48:32.315611: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-14 01:48:32.318352: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-14 01:48:32.349392: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-14 01:48:32.349426: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-14 01:48:32.349449: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

In [2]:
data_path = "/data/MHL/bepro/processed/"
results = []

if os.path.exists(data_path):
    match_id_lst = os.listdir(data_path)
    total_dict = {match_id : {} for match_id in match_id_lst}
    for match_id in match_id_lst:
        if match_id not in ["126424", "126433", "126444", "126458", "126466", "126473", "153373", "153385", "153387"]: continue
        print(f"Load {match_id}")
        with open(f"{data_path}/{match_id}/{match_id}_processed_dict.pkl", "rb") as f:
            match_dict = pickle.load(f)
        tracking_df = match_dict['tracking_df']
        event_df = match_dict['event_df']
        meta_data = match_dict['meta_data']
        teams_dict = match_dict['teams']
        with open(f"{data_path}/{match_id}/{match_id}_presing_intensity.pkl", "rb") as f:
            pressing_df = pickle.load(f)

        total_df = pd.merge(tracking_df, pressing_df, on=['game_id', 'period_id', 'timestamp', 'frame_id'], how='left')
        # total_df = _normalize_coordinate_direction(total_df, teams_dict['Home']['pID'].iloc[0])
        total_dict[match_id] = {}
        total_dict[match_id]['tracking_df'] = total_df
        total_dict[match_id]['event_df'] = event_df
        total_dict[match_id]['meta_data'] = meta_data
        total_dict[match_id]['Home'] = match_dict['teams']['Home']
        total_dict[match_id]['Away'] = match_dict['teams']['Away']


Load 126424


Load 126433
Load 126444
Load 126458
Load 126466
Load 126473
Load 153373
Load 153385
Load 153387


In [ ]:
match_id = "126424"

total_df = total_dict[match_id]['tracking_df'].copy()
event_df = total_dict[match_id]['event_df'].copy()
meta_data = total_dict[match_id]['meta_data'].copy()
home_team = total_dict[match_id]['Home'].copy()
away_team = total_dict[match_id]['Away'].copy()
teams_dict = {'Home' : home_team, 'Away' : away_team}

In [5]:
filtered_event_df = pd.read_csv(f"{data_path}/{match_id}/valid_events_filtered2.csv")
filtered_event_df.head()

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,dx,dy
0,126424,0.0,0,1,0.533,1.667,4648,408383,77414.0,-1,-1,pass,18,Successful,1.0,-1.0,0.2520,-0.3060,8.0220,-9.2820,7.7700,-8.9760
1,126424,2.0,2,1,5.167,6.500,4648,77414,145703.0,-1,-1,pass,18,Successful,3.0,-1.0,5.6280,-10.4924,16.6635,-17.5372,11.0355,-7.0448
2,126424,NaN,4,1,6.501,NaN,4648,145703,NaN,-1,-1,dribble,14,NaN,NaN,NaN,16.6635,-17.5372,16.0650,-0.0476,-0.5985,17.4896
3,126424,5.0,5,1,15.933,17.733,4648,145703,250100.0,-1,-1,pass,18,Successful,6.0,-1.0,16.0650,-0.0476,18.0495,27.4992,1.9845,27.5468
4,126424,7.0,7,1,21.433,22.700,4648,250100,250101.0,-1,-1,pass,18,Successful,8.0,-1.0,16.7475,26.4860,14.9835,10.9208,-1.7640,-15.5652


In [59]:
teams_df = pd.concat([home_team, away_team])
teams_df.reset_index(drop=True, inplace=True)
teams_df['player_code'] = teams_df.apply(lambda row : row['team'][0] + str(row['xID']).zfill(2), axis=1)
teams_df.head()

,player,position,team,jID,pID,tID,xID,player_code
0,Kyeongmin Kim,GK,Home,1,250079,4648,0,H00
1,Youngkyu Ahn,CB,Home,6,62365,4648,1,H01
2,Hyeonseok Doo,RB,Home,13,62386,4648,2,H02
3,Junsu Byeon,CB,Home,15,500138,4648,3,H03
4,Jinho Kim,LB,Home,27,500140,4648,4,H04


In [60]:
filtered_event_df = filtered_event_df.merge(
    teams_df,
    how='left',
    left_on='player_id',
    right_on='pID'
)
filtered_event_df.head()

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
0,126348,0.0,0,1,0 days 00:00:01.120000,0 days 00:00:02.360000,4657,250185,287281.0,-1,-1,pass,5,Successful,1.0,-1.0,-0.0105,0.7208,13.5870,-1.9244,13.5975,-2.6452,Changseok Song,CF,Away,91,250185,4657,10,A10
1,126348,3.0,2,1,0 days 00:00:03.280000,0 days 00:00:04.960000,4657,287281,500476.0,-1,-1,pass,5,Successful,4.0,-1.0,13.7130,0.2720,21.8400,21.1956,8.1270,20.9236,Sejong Ju,CDM,Away,8,287281,4657,5,A05
2,126348,NaN,4,1,0 days 00:00:04.960000,NaT,4657,500476,NaN,-1,-1,dribble,12,NaN,NaN,NaN,21.8400,21.1956,18.6060,22.1068,-3.2340,0.9112,Jeongtaek Lee,CB,Away,33,500476,4657,1,A01
3,126348,5.0,5,1,0 days 00:00:07.320000,0 days 00:00:08.720000,4657,500476,357146.0,-1,-1,pass,5,Successful,7.0,-1.0,18.6060,22.1068,29.0325,7.0244,10.4265,-15.0824,Jeongtaek Lee,CB,Away,33,500476,4657,1,A01
4,126348,NaN,7,1,0 days 00:00:08.720000,NaT,4657,357146,NaN,-1,-1,dribble,12,NaN,NaN,NaN,29.0325,7.0244,28.6755,-0.1496,-0.3570,-7.1740,Hyeonu Kim,CB,Away,4,357146,4657,2,A02


In [61]:
total_df.loc[total_df['period_id'] == 2, 'timestamp'] -= pd.Timedelta(minutes=45)
total_df = total_df.merge(
    teams_df[['pID', 'player_code']],
    how='left',
    left_on = 'id',
    right_on = 'pID'
)
total_df.drop(['pID'], axis=1, inplace=True)

# Ball id
nan_mask = pd.isna(total_df['player_code'])
if total_df.loc[nan_mask, 'id'].nunique() != 1:
    print('error') 
else:
    if total_df.loc[nan_mask, 'id'].unique()[0] != 'ball':
        print('error')
total_df.loc[nan_mask, 'player_code'] = total_df.loc[nan_mask, 'id']
total_df[pd.isna(total_df['player_code'])]


,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,player_code


In [62]:
# wide_tracking_df = total_df.pivot_table(
#     index= ['period_id', 'timestamp'],
#     columns='player_code',
#     values=['x', 'y', 'vx', 'vy', 'v', 'ax', 'ay', 'a']
# )

# wide_tracking_df.columns = [f'{player_code}_{value}' for value, player_code in wide_tracking_df.columns]
# wide_tracking_df.reset_index(inplace=True)
# wide_tracking_df.head()

In [94]:
total_df.head()

,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows
0,126348,1.0,0 days 00:00:01.200000,30,alive,4657,2.224286,25.2905,0.0,-0.429249,-0.411704,0.0,0.594773,-1.061013,-0.607439,0.0,1.222592,181865,4657,RWB,4657,False,"[[5.239362395384494, 4.57235841654092, 8.93770...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013..."
1,126348,1.0,0 days 00:00:01.200000,30,alive,4657,1.483500,-8.8587,0.0,-2.996945,0.537448,0.0,3.044754,-2.895288,-1.593342,0.0,3.304759,215807,4657,CF,4657,False,"[[5.239362395384494, 4.57235841654092, 8.93770...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013..."
2,126348,1.0,0 days 00:00:01.200000,30,alive,4657,-46.258071,0.5805,0.0,0.262127,-0.065138,0.0,0.270099,-0.131394,-0.356381,0.0,0.379831,250079,4648,GK,4657,False,"[[5.239362395384494, 4.57235841654092, 8.93770...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013..."
3,126348,1.0,0 days 00:00:01.200000,30,alive,4657,-10.227429,4.2540,0.0,0.518256,0.212072,0.0,0.559967,1.848757,-0.583668,0.0,1.938704,250101,4648,CM,4657,False,"[[5.239362395384494, 4.57235841654092, 8.93770...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013..."
4,126348,1.0,0 days 00:00:01.200000,30,alive,4657,-0.125679,-9.9811,0.0,0.772672,0.338248,0.0,0.843466,1.843676,0.952415,0.0,2.075147,250102,4648,CF,4657,False,"[[5.239362395384494, 4.57235841654092, 8.93770...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013..."


In [63]:
total_df = total_dict[match_id]['tracking_df'].copy()

# ball carrier에 대해 pressing intensity가 0.7보다 큰 경우 pressed_df 구성
pressed_dict = {}
ball_carrier_df = total_df[total_df['is_ball_carrier'] == True].copy()
ball_carrier_df = ball_carrier_df[ball_carrier_df['ball_state']=='alive']
ball_carrier_df.sort_values(['period_id', 'frame_id'], inplace=True)

for idx, row in tqdm(ball_carrier_df.iterrows(), desc= "Get Pressing Intensity", miniters=len(ball_carrier_df)//10):
    if len(np.where(row['rows'] == row['id'])[0]) != 0:
        pressed_axis = 'rows'
        presser_axis = 'columns'
        id_loc = np.where(row[pressed_axis] == row['id'])[0]
        # 다중 list nested 구조로 되어 있을 수 있으므로 tolist()를 두 번 적용
        pressing_values = row['probability_to_intercept'][id_loc].tolist()[0].tolist()
    elif len(np.where(row['columns'] == row['id'])[0]) != 0:
        pressed_axis = 'columns'
        presser_axis = 'rows'
        id_loc = np.where(row[pressed_axis] == row['id'])[0]
        pressing_values = [x[id_loc] for x in row['probability_to_intercept']]
    else:
        continue
    if max(pressing_values) > 0.9:
        pressed_dict[idx] = {}
        pressed_dict[idx]['pressing_value'] = max(pressing_values)
        max_idx = pressing_values.index(max(pressing_values))
        pressed_dict[idx]['pressing_player'] = row[presser_axis][max_idx]

pressed_df = ball_carrier_df.loc[list(pressed_dict.keys())].copy()
pressed_df['pressing_values'] = [d['pressing_value'] for d in pressed_dict.values()]
pressed_df['pressing_player'] = [d.get('pressing_player') for d in pressed_dict.values()]        

period_list = []
for period_id in pressed_df['period_id'].unique():
    period_df = pressed_df[pressed_df['period_id']==period_id].copy()
    # frame_id 차이가 50 프레임 이상인 경우 새로운 시퀀스로 판단 (즉, 연속된 pressed 행이 아닐 경우)
    period_df['frame_diff'] = period_df['frame_id'].diff()
    period_df['sequence_id'] = (period_df['frame_diff'] > 125).cumsum()

    # 각 시퀀스별 첫 번째 frame을 기준으로 X와 Y를 설정하기 위해 첫번째 frame 정보를 구함
    first_frames = period_df.groupby('sequence_id', as_index=False)[['timestamp', 'period_id', 'frame_id', 'id', 'team_id', 'pressing_player']].first()

    # total_df 에서 필요한 컬럼만 추출
    lookup = total_df[['period_id', 'frame_id', 'id', 'v']]

    # pressing_player 컬럼은 total_df.id 와 매칭되어 있으므로,
    # 컬럼명을 맞춰서 merge
    first_frames = first_frames.merge(
        lookup.rename(columns={'id': 'pressing_player'}),
        on=['period_id', 'frame_id', 'pressing_player'],
        how='left'
    )

    # 압박하는 선수의 속도가 2.0 m/s 인 경우만 압박으로 간주
    first_frames = first_frames[first_frames['v'] >= 2.0]
    period_list.append(first_frames)

first_frames_df = pd.concat(period_list, axis=0, ignore_index=True)


Get Pressing Intensity: 87053it [00:03, 26483.77it/s]


In [64]:
filtered_event_df['type_name'].unique()

array(['pass', 'dribble', 'recovery', 'interception', 'block', 'throw_in',
       'foul', 'freekick_crossed', 'shot', 'keeper_save', 'cross',
       'goalkick', 'clearance', 'corner_crossed', 'keeper_punch',
       'take_on'], dtype=object)

In [65]:
possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'foul', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick', 'clearance'] 

for ev in possession_gained_events:
    print(f"{ev} | {filtered_event_df[filtered_event_df['type_name']==ev]['result_name'].unique()}")

pass | ['Successful' 'Unsuccessful']
dribble | [nan]
recovery | [nan]
interception | [nan]
cross | ['Successful' 'Unsuccessful']
foul | [nan]
throw_in | ['Successful' 'Unsuccessful']
take_on | ['Unsuccessful' 'Successful']
shot | ['On Target' 'Off Target' 'Goal' 'Blocked' 'Keeper Rush-outs']
freekick_crossed | ['Successful' 'Unsuccessful']
corner_crossed | ['Unsuccessful' 'Successful']
goalkick | ['Successful' 'Unsuccessful']
clearance | [nan]


In [66]:
def check_pressing_success(row, event_df, teams_dict):
    possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick'] 
    
    if row['team_id'] == teams_dict['Home']['tID'].unique()[0]:
        pressing_team = teams_dict['Away']['tID'].unique()[0]

    elif row['team_id'] == teams_dict['Away']['tID'].unique()[0]:
        pressing_team = teams_dict['Home']['tID'].unique()[0]

    check_timegap = pd.Timedelta(seconds=5)
    window_events = event_df[
        (event_df['period_id'] == row['period_id']) &
        (event_df['time_seconds'] >= row['timestamp']) &
        (event_df['time_seconds'] <= row['timestamp'] + check_timegap)
        ]
    # event_teams = window_events['team_name'].unique()
    event_teams = window_events['tID'].unique()
    # event_team_ids = [TEAMNAME2ID[x] for x in event_teams]

    # if pressing_team in event_team_ids:
    if pressing_team in event_teams:
        # pressing_team_events = window_events[window_events['team_name'] == TEAMID2NAME[pressing_team]]
        pressing_team_events = window_events[window_events['tID'] == pressing_team]
        for _, row in pressing_team_events.iterrows():
            if row['type_name'] in possession_gained_events:
                if row['type_name'] in ['dribble', 'recovery', 'interception', 'cross', 'throw_in', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick']:
                    return True
                else: # [Pass, take_on]
                    result = row['result_name']
                    if result == 'Successful':
                        return True
        return False
    else:
        return False

first_frames_df['ball_ownership_changed'] = first_frames_df.apply(check_pressing_success, axis=1, event_df=filtered_event_df, teams_dict=teams_dict) # window=150은 예시

In [175]:
period_id = first_frames_df.iloc[0]['period_id']
frame_id = first_frames_df.iloc[0]['frame_id']

check_df = total_df[
    (total_df['period_id'] == period_id) &
    (total_df['frame_id'] == frame_id)]
timestamp = check_df['timestamp'].iloc[0]

window_event_df = filtered_event_df[
    (filtered_event_df['period_id'] == period_id) &
    (filtered_event_df['time_seconds'] >= timestamp - pd.Timedelta(seconds=5)) &
    (filtered_event_df['time_seconds'] <= timestamp)
]

In [176]:
timestamps_list = window_event_df['time_seconds'].unique().tolist() + [timestamp]
timestamps_list

[Timedelta('0 days 00:00:12.640000'),
 Timedelta('0 days 00:00:14.640000'),
 Timedelta('0 days 00:00:16')]

In [177]:
X_slice = total_df[(total_df['period_id'] == period_id) & (total_df['timestamp'].isin(timestamps_list))].copy()

In [180]:
def generate_features(frame_df: pd.DataFrame) -> pd.DataFrame:
    """
    Get additional features from tracking data

    Args:
        frame_df (pd.DataFrame): 단일 프레임에 대한 선수 및 공 데이터.
                                 'id', 'team_id', 'position_name', 'is_ball_carrier',
                                 'x', 'y', 'vx', 'vy' 등의 컬럼이 있어야 합니다.

    Returns:
        pd.DataFrame: 새로운 특징들이 추가된 데이터프레임.
    """
    # 원본 수정을 방지하기 위해 복사본 사용
    df = frame_df.copy()

    # --- 1. 주요 객체 식별 ---
    ball_row = df[df['id'] == 'ball']
    ball_carrier_row = df[df['is_ball_carrier'] == True]

    # 공 또는 공 소유자가 없는 프레임은 처리하지 않음
    if ball_row.empty or ball_carrier_row.empty:
        # 필요한 모든 특징 컬럼을 0으로 채워서 반환 (에러 방지)
        feature_cols_to_add = [
            'is_teammate', 'is_goalkeeper', 'distance_to_goal', 'sin_angle_to_goal',
            'cos_angle_to_goal', 'distance_to_ball', 'sin_angle_to_ball',
            'cos_angle_to_ball', 'cos_velocity_angle', 'sin_velocity_angle'
        ]
        for col in feature_cols_to_add:
            df[col] = 0.0
        return df

    ball_carrier = ball_carrier_row.iloc[0]
    ball = ball_row.iloc[0]
    
    # 공격 방향을 왼쪽에서 오른쪽으로 가정하고, 골대 위치를 (52.5, 0)으로 설정
    # (사전 단계에서 좌표계 통일이 되었다고 가정)
    goal_pos = np.array([52.5, 0.0])

    # --- 2. 특징 계산 ---

    # A. 이진 속성 (Binary attributes)
    # 'is_ball_carrier'는 이미 존재한다고 가정
    df['is_teammate'] = (df['team_id'] == ball_carrier['team_id']).astype(float)
    df['is_goalkeeper'] = (df['position_name'] == 'GK').astype(float)
    # is_goalpost는 실제 노드가 아니므로 특징으로 추가하지 않음

    # C. 벡터 계산을 위한 데이터 준비 (Numpy 배열)
    player_positions = df[['x', 'y']].values.astype(np.float64)
    player_velocities = df[['vx', 'vy']].values.astype(np.float64)
    ball_position = ball[['x', 'y']].values.astype(np.float64)
    carrier_velocity = ball_carrier[['vx', 'vy']].values.astype(np.float64)

    # D. 골대 기준 상대적 특징 (Relative to goalpost)
    vector_to_goal = goal_pos - player_positions
    df['distance_to_goal'] = np.linalg.norm(vector_to_goal, axis=1)
    angle_to_goal_rad = np.arctan2(vector_to_goal[:, 1], vector_to_goal[:, 0])
    df['sin_angle_to_goal'] = np.sin(angle_to_goal_rad)
    df['cos_angle_to_goal'] = np.cos(angle_to_goal_rad)

    # E. 공 기준 상대적 특징 (Relative to ball)
    vector_to_ball = ball_position - player_positions
    df['distance_to_ball'] = np.linalg.norm(vector_to_ball, axis=1)
    angle_to_ball_rad = np.arctan2(vector_to_ball[:, 1], vector_to_ball[:, 0])
    df['sin_angle_to_ball'] = np.sin(angle_to_ball_rad)
    df['cos_angle_to_ball'] = np.cos(angle_to_ball_rad)

    # F. 선수 속도 벡터와 공 소유자 속도 벡터 간의 각도
    dot_product = np.sum(player_velocities * carrier_velocity, axis=1)
    norm_player = np.linalg.norm(player_velocities, axis=1)
    norm_carrier = np.linalg.norm(carrier_velocity)
    
    # 0으로 나누는 것을 방지하기 위한 작은 값(epsilon)
    denominator = (norm_player * norm_carrier) + 1e-8
    
    df['cos_velocity_angle'] = np.clip(dot_product / denominator, -1.0, 1.0)
    
    # 2D 벡터의 외적(cross product)을 이용한 사인 값 계산
    cross_product = player_velocities[:, 0] * carrier_velocity[1] - player_velocities[:, 1] * carrier_velocity[0]
    df['sin_velocity_angle'] = np.clip(cross_product / denominator, -1.0, 1.0)
    
    return df

In [184]:
X_slice_feats = X_slice.set_index('frame_id').groupby('frame_id', group_keys=False).apply(generate_features)
X_slice_feats.reset_index()

,frame_id,game_id,period_id,timestamp,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
0,316,126348,1.0,0 days 00:00:12.640000,alive,4657,-3.481925,29.298045,0.0,-0.176889,0.139786,0.0,0.225455,1.684517,1.309493,0.0,2.133629,181865,4657,RWB,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,63.185056,-0.463686,0.885999,43.282374,-0.668294,0.743898,0.353768,-0.935333
1,316,126348,1.0,0 days 00:00:12.640000,alive,4657,-2.099240,-3.007794,0.0,1.007120,1.847529,0.0,2.104199,-1.020331,4.492951,0.0,4.607351,215807,4657,CF,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,54.682024,0.055005,0.998486,30.999837,0.109049,0.994036,0.981640,0.190744
2,316,126348,1.0,0 days 00:00:12.640000,alive,4657,-39.429641,-0.754456,0.0,-0.212315,0.669514,0.0,0.702372,-0.072863,1.305936,0.0,1.307967,250079,4648,GK,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,1.0,91.932737,0.008207,0.999966,68.154688,0.016538,0.999863,0.817204,-0.576348
3,316,126348,1.0,0 days 00:00:12.640000,alive,4657,12.986974,4.586908,0.0,0.672207,1.930585,0.0,2.044265,-2.197860,2.893080,0.0,3.633249,250101,4648,CM,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,39.778373,-0.115312,0.993329,16.283523,-0.258802,0.965931,0.999611,0.027889
4,316,126348,1.0,0 days 00:00:12.640000,alive,4657,17.656188,-1.548861,0.0,0.814706,2.277274,0.0,2.418620,-0.261490,2.064434,0.0,2.080929,250102,4648,CF,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,34.878220,0.044408,0.999013,11.225231,0.171183,0.985239,0.999338,0.036392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,400,126348,1.0,0 days 00:00:16,alive,4657,3.447429,-7.841800,0.0,0.645528,3.402015,0.0,3.462717,0.089743,-0.937323,0.0,0.941610,500493,4657,CM,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,49.675432,0.157861,0.987461,41.479184,0.824447,0.565939,0.006796,0.999977
65,400,126348,1.0,0 days 00:00:16,alive,4657,-6.694071,13.515200,0.0,-0.426502,3.024344,0.0,3.054269,-0.605294,-0.382521,0.0,0.716033,62365,4648,CB,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,60.717368,-0.222592,0.974912,35.985043,0.356826,0.934171,0.328216,0.944603
66,400,126348,1.0,0 days 00:00:16,alive,4657,-10.920643,-15.593000,0.0,0.080653,3.318039,0.0,3.319020,0.539804,0.279326,0.0,0.607792,62386,4648,RB,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 2

In [182]:
X_slice_feats

,game_id,period_id,timestamp,frame_id,ball_state,ori_ball_owning_team_id,x,y,z,vx,vy,vz,v,ax,ay,az,a,id,team_id,position_name,ball_owning_team_id,is_ball_carrier,time_to_intercept,probability_to_intercept,columns,rows,is_teammate,is_goalkeeper,distance_to_goal,sin_angle_to_goal,cos_angle_to_goal,distance_to_ball,sin_angle_to_ball,cos_angle_to_ball,cos_velocity_angle,sin_velocity_angle
6578,126348,1.0,0 days 00:00:12.640000,316,alive,4657,-3.481925,29.298045,0.0,-0.176889,0.139786,0.0,0.225455,1.684517,1.309493,0.0,2.133629,181865,4657,RWB,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,63.185056,-0.463686,0.885999,43.282374,-0.668294,0.743898,0.353768,-0.935333
6579,126348,1.0,0 days 00:00:12.640000,316,alive,4657,-2.099240,-3.007794,0.0,1.007120,1.847529,0.0,2.104199,-1.020331,4.492951,0.0,4.607351,215807,4657,CF,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,54.682024,0.055005,0.998486,30.999837,0.109049,0.994036,0.981640,0.190744
6580,126348,1.0,0 days 00:00:12.640000,316,alive,4657,-39.429641,-0.754456,0.0,-0.212315,0.669514,0.0,0.702372,-0.072863,1.305936,0.0,1.307967,250079,4648,GK,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,1.0,91.932737,0.008207,0.999966,68.154688,0.016538,0.999863,0.817204,-0.576348
6581,126348,1.0,0 days 00:00:12.640000,316,alive,4657,12.986974,4.586908,0.0,0.672207,1.930585,0.0,2.044265,-2.197860,2.893080,0.0,3.633249,250101,4648,CM,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,39.778373,-0.115312,0.993329,16.283523,-0.258802,0.965931,0.999611,0.027889
6582,126348,1.0,0 days 00:00:12.640000,316,alive,4657,17.656188,-1.548861,0.0,0.814706,2.277274,0.0,2.418620,-0.261490,2.064434,0.0,2.080929,250102,4648,CF,4657,False,"[[4.676868636714823, 5.224540929733676, 8.6782...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,34.878220,0.044408,0.999013,11.225231,0.171183,0.985239,0.999338,0.036392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8528,126348,1.0,0 days 00:00:16,400,alive,4657,3.447429,-7.841800,0.0,0.645528,3.402015,0.0,3.462717,0.089743,-0.937323,0.0,0.941610,500493,4657,CM,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",1.0,0.0,49.675432,0.157861,0.987461,41.479184,0.824447,0.565939,0.006796,0.999977
8529,126348,1.0,0 days 00:00:16,400,alive,4657,-6.694071,13.515200,0.0,-0.426502,3.024344,0.0,3.054269,-0.605294,-0.382521,0.0,0.716033,62365,4648,CB,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[181865, 215807, 250167, 250172, 250175, 25018...","[250079, 250101, 250102, 360166, 500136, 50013...",0.0,0.0,60.717368,-0.222592,0.974912,35.985043,0.356826,0.934171,0.328216,0.944603
8530,126348,1.0,0 days 00:00:16,400,alive,4657,-10.920643,-15.593000,0.0,0.080653,3.318039,0.0,3.319020,0.539804,0.279326,0.0,0.607792,62386,4648,RB,4657,False,"[[9.129624832393825, 6.760512075950806, 8.3164...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[1818

In [53]:
from IPython.display import display

row = first_frames_df.iloc[3]

possession_gained_events = ['pass', 'dribble', 'recovery', 'interception', 'cross', 'throw_in', 'take_on', 'shot', 'freekick_crossed', 'corner_crossed', 'goalkick'] 

if row['team_id'] == teams_dict['Home']['tID'].unique()[0]:
    pressing_team = teams_dict['Away']['tID'].unique()[0]

elif row['team_id'] == teams_dict['Away']['tID'].unique()[0]:
    pressing_team = teams_dict['Home']['tID'].unique()[0]

print(f"Pressing Team : {pressing_team}")
print(f"Pressing Time : {row['timestamp']}")
print(f"Results : {row['ball_ownership_changed']}")

check_timegap = pd.Timedelta(seconds=5)

prev_window_events = filtered_event_df[
        (filtered_event_df['period_id'] == row['period_id']) &
        (filtered_event_df['time_seconds'] >= row['timestamp'] - pd.Timedelta(seconds=5)) &
        (filtered_event_df['time_seconds'] <= row['timestamp'])
        ].tail(5)

aft_window_events = filtered_event_df[
        (filtered_event_df['period_id'] == row['period_id']) &
        (filtered_event_df['time_seconds'] >= row['timestamp']) &
        (filtered_event_df['time_seconds'] <= row['timestamp'] + check_timegap)
        ]
# event_teams = window_events['team_name'].unique()
event_teams = aft_window_events['tID'].unique()
# event_team_ids = [TEAMNAME2ID[x] for x in event_teams]

print("Before Pressing")
display(prev_window_events[['period_id', 'time_seconds', 'team_id', 'player_id', 'type_name', 'result_name']])

print("After Pressing")
display(aft_window_events[['period_id', 'time_seconds', 'team_id','player_id', 'type_name', 'result_name']])

Pressing Team : 4648
Pressing Time : 0 days 00:01:37.920000
Results : False
Before Pressing


,period_id,time_seconds,team_id,player_id,type_name,result_name
33,1,0 days 00:01:36.120000,4640,491837.0,freekick_crossed,Unsuccessful
34,1,0 days 00:01:36.320000,4648,500134.0,block,NaN
35,1,0 days 00:01:36.720000,4640,491837.0,recovery,NaN


After Pressing


,period_id,time_seconds,team_id,player_id,type_name,result_name
36,1,0 days 00:01:38.360000,4640,491837.0,pass,Successful
37,1,0 days 00:01:40.080000,4640,354791.0,pass,Successful


In [293]:
total_df[
    (total_df['period_id'] == row['period_id']) &
    (total_df['timestamp'].isin(prev_window_events['time_seconds'].unique().tolist() + [row['timestamp']]))
    ].shape

(69, 26)

In [220]:
first_frames_df['ball_ownership_changed'].value_counts()

ball_ownership_changed
False    136
True      58
Name: count, dtype: int64

In [97]:
time_window = pd.Timedelta(seconds=5)
row = first_frames_df.iloc[2]
ref_period_id = row['period_id']
ref_timestamp = row['timestamp']

start_time = ref_timestamp - time_window
end_time = ref_timestamp + time_window

filtered_event_df[
    (filtered_event_df['period_id'] == ref_period_id) &
    # (filtered_event_df['time_seconds'] >= start_time) &
    (filtered_event_df['time_seconds'] <= ref_timestamp)
    ]

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
0,126315,0.0,0,1,0 days 00:00:02.160000,0 days 00:00:03.960000,4648,360389,62365.0,-1,-1,pass,4,Successful,4.0,-1.0,52.5840,35.8428,69.0480,46.8860,69.0480,46.8860,16.4640,11.0432,Kang Hyun Lee,CM,Away,24,360389,4648,6,A06
1,126315,5.0,2,1,0 days 00:00:04.640000,0 days 00:00:06,4648,62365,500133.0,-1,-1,pass,4,Successful,6.0,-1.0,68.9430,47.7428,69.8565,31.0964,69.8565,31.0964,0.9135,-16.6464,Youngkyu Ahn,CB,Away,6,62365,4648,3,A03
2,126315,NaN,4,1,0 days 00:00:06,NaT,4648,500133,NaN,-1,-1,dribble,2,NaN,NaN,NaN,69.8565,31.0964,57.4665,29.8928,NaN,NaN,-12.3900,-1.2036,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
3,126315,8.0,5,1,0 days 00:00:10.680000,0 days 00:00:12.440000,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
4,126315,9.0,7,1,0 days 00:00:12.440000,0 days 00:00:13.520000,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388,Mingi Lee,LB,Away,3,145701,4648,1,A01


In [93]:
filtered_event_df.columns

Index(['game_id', 'original_event_id', 'action_id', 'period_id',
       'time_seconds', 'relative_time_seconds', 'team_id', 'player_id',
       'relative_player_id', 'reactor_team_id', 'reactor_player_id',
       'type_name', 'type_id', 'result_name', 'relative_id', 'pair_id',
       'start_x', 'start_y', 'end_x', 'end_y', 'dx', 'dy', 'player',
       'position', 'team', 'jID', 'pID', 'tID', 'xID', 'player_code'],
      dtype='object')

In [90]:
filtered_event_df[
    (filtered_event_df['period_id'] == ref_period_id) &
    (filtered_event_df['time_seconds'] >= ref_timestamp) &
    (filtered_event_df['time_seconds'] <= end_time)
    ]

,game_id,original_event_id,action_id,period_id,time_seconds,relative_time_seconds,team_id,player_id,relative_player_id,reactor_team_id,reactor_player_id,type_name,type_id,result_name,relative_id,pair_id,start_x,start_y,end_x,end_y,relative_x,relative_y,dx,dy,player,position,team,jID,pID,tID,xID,player_code
3,126315,8.0,5,1,0 days 00:00:10.680000,0 days 00:00:12.440000,4648,500133,145701.0,-1,-1,pass,4,Successful,9.0,-1.0,57.4665,29.8928,59.6820,15.9664,59.6820,15.9664,2.2155,-13.9264,Alexandar Popovic,CB,Away,4,500133,4648,2,A02
4,126315,9.0,7,1,0 days 00:00:12.440000,0 days 00:00:13.520000,4648,145701,77414.0,-1,-1,pass,4,Successful,11.0,-1.0,59.6820,15.9664,41.8845,14.3276,41.8845,14.3276,-17.7975,-1.6388,Mingi Lee,LB,Away,3,145701,4648,1,A01
5,126315,NaN,9,1,0 days 00:00:13.520000,NaT,4648,77414,NaN,-1,-1,dribble,2,NaN,NaN,NaN,41.8845,14.3276,35.0595,14.4636,NaN,NaN,-6.8250,0.1360,Huigyun Lee,CF,Away,10,77414,4648,9,A09
6,126315,12.0,12,1,0 days 00:00:14.480000,NaT,4648,250102,-1.0,-1,-1,recovery,15,NaN,-1.0,13.0,35.0595,14.4636,35.0595,14.4636,NaN,NaN,0.0000,0.0000,Kunhee Lee,CF,Away,20,250102,4648,10,A10
7,126315,NaN,13,1,0 days 00:00:14.520000,NaT,4648,250102,NaN,-1,-1,dribble,2,NaN,NaN,NaN,35.0595,14.4636,38.9970,24.6500,NaN,NaN,3.9375,10.1864,Kunhee Lee,CF,Away,20,250102,4648,10,A10
